## Image Interpolation

In [1]:
import cv2
import os
import numpy as np

# Paths
input_folder = "stage3"
output_folder = "stage3-interpolated"
os.makedirs(output_folder, exist_ok=True)

def interpolate_frames(frame1, frame2, num_interpolations=2):
    """
    Interpolates between two frames using linear interpolation.
    
    Args:
        frame1 (numpy.ndarray): First frame.
        frame2 (numpy.ndarray): Second frame.
        num_interpolations (int): Number of interpolated frames to generate.
    
    Returns:
        list: List of interpolated frames.
    """
    interpolated_frames = []
    for i in range(1, num_interpolations + 1):
        alpha = i / (num_interpolations + 1)  # Interpolation ratio
        interpolated_frame = cv2.addWeighted(frame1, 1 - alpha, frame2, alpha, 0)
        interpolated_frames.append(interpolated_frame)
    return interpolated_frames

def process_video(video_path, output_path, num_interpolations=2):
    """
    Processes a video to add interpolated frames and saves the result.
    
    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the interpolated video.
        num_interpolations (int): Number of interpolated frames between each pair of original frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video: {video_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
    
    # New FPS after interpolation
    new_fps = fps * (num_interpolations + 1)
    out = cv2.VideoWriter(output_path, fourcc, new_fps, (width, height))
    
    ret, prev_frame = cap.read()
    while ret:
        ret, next_frame = cap.read()
        if not ret:
            break

        # Write the original frame
        out.write(prev_frame)

        # Generate and write interpolated frames
        interpolated_frames = interpolate_frames(prev_frame, next_frame, num_interpolations)
        for frame in interpolated_frames:
            out.write(frame)
        
        # Update the previous frame
        prev_frame = next_frame

    # Release resources
    cap.release()
    out.release()
    print(f"Interpolated video saved to: {output_path}")

# Process videos inside each user's folder
num_interpolations = 2  # Number of interpolated frames between each pair

for user_folder in os.listdir(input_folder):
    user_folder_path = os.path.join(input_folder, user_folder)
    if os.path.isdir(user_folder_path):
        user_output_folder = os.path.join(output_folder, user_folder)
        os.makedirs(user_output_folder, exist_ok=True)

        for video_file in ["a.mp4", "e.mp4"]:
            input_path = os.path.join(user_folder_path, video_file)
            output_path = os.path.join(user_output_folder, video_file)
            
            if os.path.exists(input_path):
                process_video(input_path, output_path, num_interpolations=num_interpolations)
            else:
                print(f"Video {video_file} not found in {user_folder}")

print("Interpolation complete for all videos.")


Interpolated video saved to: stage3-interpolated\user1\a.mp4
Interpolated video saved to: stage3-interpolated\user1\e.mp4
Interpolated video saved to: stage3-interpolated\user10\a.mp4
Interpolated video saved to: stage3-interpolated\user10\e.mp4
Interpolated video saved to: stage3-interpolated\user12\a.mp4
Interpolated video saved to: stage3-interpolated\user12\e.mp4
Interpolated video saved to: stage3-interpolated\user13\a.mp4
Interpolated video saved to: stage3-interpolated\user13\e.mp4
Interpolated video saved to: stage3-interpolated\user19\a.mp4
Interpolated video saved to: stage3-interpolated\user19\e.mp4
Interpolated video saved to: stage3-interpolated\user2\a.mp4
Interpolated video saved to: stage3-interpolated\user2\e.mp4
Interpolated video saved to: stage3-interpolated\user20\a.mp4
Interpolated video saved to: stage3-interpolated\user20\e.mp4
Interpolated video saved to: stage3-interpolated\user22\a.mp4
Interpolated video saved to: stage3-interpolated\user22\e.mp4
Interpolated

In [4]:
import cv2
import os

# Path to the folder containing the videos
stage_path = "stage3-interpolated"

# Dictionary to store frame counts for 1.mp4 and 2.mp4 separately
frame_counts = {"e.mp4": [], "a.mp4": []}

# Iterate through user folders
for user_folder in os.listdir(stage_path):
    user_folder_path = os.path.join(stage_path, user_folder)
    
    if os.path.isdir(user_folder_path):
        for video_file in ["e.mp4", "a.mp4"]:
            video_path = os.path.join(user_folder_path, video_file)

            # Check if video exists in the user's folder
            if os.path.exists(video_path):
                cap = cv2.VideoCapture(video_path)
                
                if not cap.isOpened():
                    print(f"Error opening video file: {video_path}")
                    continue

                # Get the total number of frames
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                frame_counts[video_file].append(total_frames)
                cap.release()
            else:
                print(f"Video {video_file} not found in {user_folder_path}")

# Calculate statistics for 1.mp4 and 2.mp4
for video_file in frame_counts:
    if frame_counts[video_file]:
        avg_frames = sum(frame_counts[video_file]) / len(frame_counts[video_file])
        min_frames = min(frame_counts[video_file])
        max_frames = max(frame_counts[video_file])

        print(f"Statistics for {video_file}:")
        print(f"  Average frames: {avg_frames:.2f}")
        print(f"  Minimum frames: {min_frames}")
        print(f"  Maximum frames: {max_frames}")
    else:
        print(f"No valid videos found for {video_file}.")


Statistics for e.mp4:
  Average frames: 27.00
  Minimum frames: 27
  Maximum frames: 27
Statistics for a.mp4:
  Average frames: 132.23
  Minimum frames: 69
  Maximum frames: 234


## Standardize videos

In [5]:
import cv2
import os

# Paths to input and output directories
input_dir = "stage3-interpolated"
output_dir = "stage3-processed"
os.makedirs(output_dir, exist_ok=True)

# Target number of frames for each video
target_frames_dict = {
    "a.mp4": 130,
    "e.mp4": 27
}

def truncate_or_pad_video(video_path, output_path, target_frames):
    """
    Truncate or pad a video to ensure it has exactly `target_frames` frames.

    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the processed video.
        target_frames (int): Desired number of frames.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Create VideoWriter for the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    if len(frames) < target_frames:
        # Pad by duplicating the last frame
        while len(frames) < target_frames:
            frames.append(frames[-1])
    elif len(frames) > target_frames:
        # Truncate to the required number of frames
        frames = frames[:target_frames]

    # Write the processed frames to the output video
    for frame in frames:
        out.write(frame)
    out.release()

# Process all videos in each user's folder
for user_folder in os.listdir(input_dir):
    user_input_folder = os.path.join(input_dir, user_folder)
    user_output_folder = os.path.join(output_dir, user_folder)
    os.makedirs(user_output_folder, exist_ok=True)

    if os.path.isdir(user_input_folder):
        for video_file, target_frames in target_frames_dict.items():
            input_path = os.path.join(user_input_folder, video_file)
            output_path = os.path.join(user_output_folder, video_file)

            if os.path.exists(input_path):
                truncate_or_pad_video(input_path, output_path, target_frames)
                print(f"Processed {video_file} in {user_folder} to {target_frames} frames")
            else:
                print(f"Video {video_file} not found in {user_folder}")

print("All videos processed and standardized.")


Processed a.mp4 in user1 to 130 frames
Processed e.mp4 in user1 to 27 frames
Processed a.mp4 in user10 to 130 frames
Processed e.mp4 in user10 to 27 frames
Processed a.mp4 in user12 to 130 frames
Processed e.mp4 in user12 to 27 frames
Processed a.mp4 in user13 to 130 frames
Processed e.mp4 in user13 to 27 frames
Processed a.mp4 in user19 to 130 frames
Processed e.mp4 in user19 to 27 frames
Processed a.mp4 in user2 to 130 frames
Processed e.mp4 in user2 to 27 frames
Processed a.mp4 in user20 to 130 frames
Processed e.mp4 in user20 to 27 frames
Processed a.mp4 in user22 to 130 frames
Processed e.mp4 in user22 to 27 frames
Processed a.mp4 in user23 to 130 frames
Processed e.mp4 in user23 to 27 frames
Processed a.mp4 in user3 to 130 frames
Processed e.mp4 in user3 to 27 frames
Processed a.mp4 in user5 to 130 frames
Processed e.mp4 in user5 to 27 frames
Processed a.mp4 in user6 to 130 frames
Processed e.mp4 in user6 to 27 frames
Processed a.mp4 in user8 to 130 frames
Processed e.mp4 in user

## Augmentation

In [7]:
import cv2
import os
import pandas as pd
import numpy as np

# Paths
input_folder = "stage3-processed"
output_folder = "stage3-dataset"
os.makedirs(output_folder, exist_ok=True)

# Load labels
labels_df = pd.read_csv("stage3.csv")

# Augmentation Functions
def augment_video(video_path, output_path, augmentation_type, is_odd):
    """
    Augment the video with the specified augmentation type and save it.
    
    Args:
        video_path (str): Path to the input video.
        output_path (str): Path to save the augmented video.
        augmentation_type (str): Type of augmentation ('rotate', 'brightness', 'noise', etc.).
        is_odd (bool): Whether the video index is odd or even.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files

    # Create VideoWriter object
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply augmentation
        if augmentation_type == "mirrored":
            frame = cv2.flip(frame, 1)  # Horizontal flip
        elif augmentation_type == "rotate":
            angle = 3 if is_odd else -3  # Adjusted rotation angles
            M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
            frame = cv2.warpAffine(frame, M, (width, height))
        elif augmentation_type == "brightness":
            alpha = 1.05 if is_odd else 0.95  # Adjusted brightness
            frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=0)
        elif augmentation_type == "noise":
            noise = np.random.normal(0, 15, frame.shape).astype(np.uint8)  # Add small random noise
            frame = cv2.add(frame, noise)

        # Write the augmented frame
        out.write(frame)

    cap.release()
    out.release()

def combine_videos(video1_path, video2_path, output_path):
    """
    Combines two videos by concatenating their frames.
    
    Args:
        video1_path (str): Path to the first video (e.mp4).
        video2_path (str): Path to the second video (a.mp4).
        output_path (str): Path to save the combined video.
    """
    cap1 = cv2.VideoCapture(video1_path)
    cap2 = cv2.VideoCapture(video2_path)

    if not cap1.isOpened() or not cap2.isOpened():
        print(f"Error: Unable to open one of the videos: {video1_path} or {video2_path}")
        return

    width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap1.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap1.isOpened():
        ret, frame = cap1.read()
        if not ret:
            break
        out.write(frame)

    while cap2.isOpened():
        ret, frame = cap2.read()
        if not ret:
            break
        out.write(frame)

    cap1.release()
    cap2.release()
    out.release()

# Augment videos and create a new CSV file
augmented_data = []
for idx, row in labels_df.iterrows():
    user_folder = row['video']
    label = row['label']

    video_e_path = os.path.join(input_folder, user_folder, "e.mp4")
    video_a_path = os.path.join(input_folder, user_folder, "a.mp4")
    
    if not os.path.exists(video_e_path) or not os.path.exists(video_a_path):
        print(f"Skipping user {user_folder} due to missing videos.")
        continue

    combined_video_path = os.path.join(output_folder, f"{user_folder}_original.mp4")
    combine_videos(video_e_path, video_a_path, combined_video_path)

    augmented_data.append({
        "video": combined_video_path,
        "label": label
    })

    # Apply augmentations
    is_odd = (idx % 2 == 1)
    for aug_type in ["mirrored", "rotate", "brightness", "noise"]:
        output_video_name = f"{user_folder}_{aug_type}.mp4"
        output_video_path = os.path.join(output_folder, output_video_name)
        augment_video(combined_video_path, output_video_path, aug_type, is_odd)

        # Append augmented video to dataset
        augmented_data.append({
            "video": output_video_path,
            "label": label
        })

# Create a new DataFrame for the augmented dataset
augmented_df = pd.DataFrame(augmented_data)

# Save the new dataset labels to a CSV file
augmented_df.to_csv("stage3-dataset.csv", index=False)
print("Augmented videos saved to stage3-dataset and labels saved to stage3-dataset.csv.")


Augmented videos saved to stage3-dataset and labels saved to stage3-dataset.csv.


## Keypoints extraction

In [9]:
import os
import cv2
import mediapipe as mp
import numpy as np
import json

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calculate_angle(a, b, c):
    """
    Calculate the angle formed by three points (a, b, c) where 'b' is the vertex.
    """
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Paths
input_videos_path = "stage3-dataset"  # Path to input videos
output_json_path = "stage3-dataset-json"  # Path to save JSON files
os.makedirs(output_json_path, exist_ok=True)

# Process all videos in the input directory
for file in os.listdir(input_videos_path):
    if file.endswith(".mp4"):
        video_file_path = os.path.join(input_videos_path, file)
        cap = cv2.VideoCapture(video_file_path)

        keypoints_data = []
        frame_count = 0
        processed_frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Skip every second frame
            if frame_count % 2 == 0:
                continue

            processed_frame_count += 1

            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = pose.process(frame_rgb)

            if result.pose_landmarks:
                landmarks = result.pose_landmarks.landmark

                # Extract keypoints for analysis
                nose = [landmarks[mp_pose.PoseLandmark.NOSE].x, landmarks[mp_pose.PoseLandmark.NOSE].y]
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_HIP].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]

                # Calculate body angles and movement
                head_angle = calculate_angle(left_shoulder, nose, right_shoulder)
                hip_movement = right_hip[0] - left_hip[0]  # Horizontal movement tracking

                # Determine whether the frame belongs to baton exchange or not
                is_exchange = 1 if processed_frame_count <= 27 else 0

                # Store extracted features
                keypoints_data.append({
                    "frame": processed_frame_count,
                    "is_exchange": is_exchange,
                    "head_angle": head_angle,
                    "hip_movement": hip_movement,
                    "nose_x": nose[0],
                    "nose_y": nose[1],
                    "left_shoulder_x": left_shoulder[0],
                    "left_shoulder_y": left_shoulder[1],
                    "right_shoulder_x": right_shoulder[0],
                    "right_shoulder_y": right_shoulder[1],
                    "left_hip_x": left_hip[0],
                    "left_hip_y": left_hip[1],
                    "right_hip_x": right_hip[0],
                    "right_hip_y": right_hip[1]
                })

        cap.release()

        # Save keypoints to a JSON file
        json_filename = os.path.splitext(file)[0] + "_keypoints.json"
        json_file_path = os.path.join(output_json_path, json_filename)
        with open(json_file_path, "w") as json_file:
            json.dump(keypoints_data, json_file, indent=4)

        print(f"Processed: {file} - Keypoints saved to {json_file_path}")

print("All video processing completed. Keypoints saved as JSON.")


Processed: user10_brightness.mp4 - Keypoints saved to stage3-dataset-json\user10_brightness_keypoints.json
Processed: user10_mirrored.mp4 - Keypoints saved to stage3-dataset-json\user10_mirrored_keypoints.json
Processed: user10_noise.mp4 - Keypoints saved to stage3-dataset-json\user10_noise_keypoints.json
Processed: user10_original.mp4 - Keypoints saved to stage3-dataset-json\user10_original_keypoints.json
Processed: user10_rotate.mp4 - Keypoints saved to stage3-dataset-json\user10_rotate_keypoints.json
Processed: user12_brightness.mp4 - Keypoints saved to stage3-dataset-json\user12_brightness_keypoints.json
Processed: user12_mirrored.mp4 - Keypoints saved to stage3-dataset-json\user12_mirrored_keypoints.json
Processed: user12_noise.mp4 - Keypoints saved to stage3-dataset-json\user12_noise_keypoints.json
Processed: user12_original.mp4 - Keypoints saved to stage3-dataset-json\user12_original_keypoints.json
Processed: user12_rotate.mp4 - Keypoints saved to stage3-dataset-json\user12_rota

In [4]:
import os
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Paths
json_folder = "stage3-dataset-json"  # Folder containing JSON files with extracted features
labels_file = "stage3-dataset.csv"  # CSV file with video labels (e.g., user1_original.mp4: 0)

# Load labels (assumes a CSV file with 'video' and 'label' columns)
labels_df = pd.read_csv(labels_file)
label_mapping = dict(zip(labels_df['video'], labels_df['label']))

# Function to calculate distance between two points
def calculate_distance(p1, p2):
    return np.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

# Function to calculate angles between three points
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Dataset preparation
X = []  # Features
y = []  # Labels
max_sequence_length = 250  # Fixed sequence length for LSTM

for json_file in sorted(os.listdir(json_folder)):
    video_name = os.path.splitext(json_file)[0] + ".mp4"  # Get full video filename
    if video_name not in label_mapping:
        print(f"Label not found for {video_name}. Skipping...")
        continue

    label = label_mapping[video_name]  # Get label for the video
    json_path = os.path.join(json_folder, json_file)

    # Load JSON data
    with open(json_path, "r") as file:
        data = json.load(file)

    # Extract meaningful features
    video_features = []
    for frame in data:
        try:
            # Extract coordinates
            nose = [frame["nose_x"], frame["nose_y"]]
            left_shoulder = [frame["left_shoulder_x"], frame["left_shoulder_y"]]
            right_shoulder = [frame["right_shoulder_x"], frame["right_shoulder_y"]]
            left_hip = [frame["left_hip_x"], frame["left_hip_y"]]
            right_hip = [frame["right_hip_x"], frame["right_hip_y"]]

            # Compute meaningful features
            shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
            hip_distance = calculate_distance(left_hip, right_hip)
            torso_length = calculate_distance(left_shoulder, left_hip)

            # Shoulder-hip and head tilt angles
            shoulder_hip_angle = calculate_angle(left_shoulder, left_hip, right_hip)
            head_tilt_angle = calculate_angle(left_shoulder, nose, right_shoulder)

            # Store reduced features
            features = [
                shoulder_distance,  # Horizontal upper body width
                hip_distance,       # Horizontal lower body width
                torso_length,       # Vertical body height
                shoulder_hip_angle, # Posture angle
                head_tilt_angle,    # Head tilt angle
                frame["is_exchange"]  # Frame label for exchange period
            ]
            video_features.append(features)
        except KeyError as e:
            print(f"Missing key {e} in {json_file}. Skipping frame...")
            continue

    # Append features and label if data is available
    if video_features:
        X.append(video_features)
        y.append(label)

# Pad sequences to ensure consistent lengths
X = pad_sequences(X, maxlen=max_sequence_length, dtype='float32', padding='post', truncating='post')

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Model Constants
sequence_length = X_train.shape[1]  # Fixed length of sequences (250 frames)
num_features = X_train.shape[2]  # Number of reduced features per frame
num_classes = len(np.unique(y_train))  # Number of unique labels

print(f"Dataset prepared:")
print(f"Training set shape: {X_train.shape}, Validation set shape: {X_val.shape}")
print(f"Number of features per frame: {num_features}")
print(f"Number of classes: {num_classes}")


Dataset prepared:
Training set shape: (42, 250, 6), Validation set shape: (18, 250, 6)
Number of features per frame: 6
Number of classes: 2


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.compat.v1.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calculate class weights to handle data imbalance
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.unique(y_train), 
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")

# Build the LSTM model
model = Sequential([
    LSTM(128, activation='tanh', return_sequences=True, input_shape=(sequence_length, num_features)),
    Dropout(0.3),
    LSTM(64, activation='tanh', return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Output: Value between 0 and 1 for binary classification
])

# Compile the model with Adam optimizer and binary crossentropy loss
model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='binary_crossentropy',  # Using binary crossentropy for binary classification
    metrics=['accuracy']  # Train on accuracy metric
)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with class weights
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # Adjust based on dataset size
    batch_size=32,  # Adjust based on memory constraints
    class_weight=class_weights_dict,  # Use class weights to address imbalance
    callbacks=[early_stopping],  # Early stopping to prevent overfitting
    verbose=1
)

# Save the trained model
model.save("stage3-final.keras")
print("Model training complete and saved as 'stage3-final.keras'")

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Accuracy: {val_accuracy:.4f}")


Class weights: {0: 0.6774193548387096, 1: 1.9090909090909092}
Epoch 1/100
2/2 [==============================] - 10s 4s/step - loss: 0.6932 - accuracy: 0.7381 - val_loss: 0.6930 - val_accuracy: 0.7778
Epoch 2/100
2/2 [==============================] - 6s 3s/step - loss: 0.6932 - accuracy: 0.7381 - val_loss: 0.6932 - val_accuracy: 0.2222
Epoch 3/100
2/2 [==============================] - 5s 3s/step - loss: 0.6931 - accuracy: 0.2619 - val_loss: 0.6933 - val_accuracy: 0.2222
Epoch 4/100
2/2 [==============================] - 5s 3s/step - loss: 0.6932 - accuracy: 0.2619 - val_loss: 0.6933 - val_accuracy: 0.2222
Epoch 5/100
2/2 [==============================] - 6s 3s/step - loss: 0.6931 - accuracy: 0.2619 - val_loss: 0.6932 - val_accuracy: 0.2222
Epoch 6/100
2/2 [==============================] - 6s 3s/step - loss: 0.6931 - accuracy: 0.2619 - val_loss: 0.6931 - val_accuracy: 0.7778
Epoch 7/100
2/2 [==============================] - 6s 3s/step - loss: 0.6932 - accuracy: 0.7381 - val_loss: 0